In [ ]:
#!pip install matplotlib tensorflow-gpu tensorflow-hub

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from matplotlib.pyplot import imsave
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
def superResolutionDo(img_path):
    lr = tf.io.read_file(img_path)
    lr = tf.image.decode_jpeg(lr)
    lr = tf.expand_dims(lr, axis=0)
    lr = tf.cast(lr, tf.float32)

    model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")


    concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
    concrete_func.inputs[0].set_shape(lr.shape.as_list())
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Save the TF Lite model.
    with tf.io.gfile.GFile('ESRGAN.tflite', 'wb') as f:
      f.write(tflite_model)

    esrgan_model_path = './ESRGAN.tflite'


    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=esrgan_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Run the model
    interpreter.set_tensor(input_details[0]['index'], lr)
    interpreter.invoke()

    # Extract the output and postprocess it
    output_data = interpreter.get_tensor(output_details[0]['index'])
    sr = tf.squeeze(output_data, axis=0)
    sr = tf.clip_by_value(sr, 0, 255)
    sr = tf.round(sr)
    sr = tf.cast(sr, tf.uint8)
    return sr.numpy()

# # test_img_path="/prj/nowage/ImgAugWithSR/imgs/HardHatSample__small/images/000101_jpg.rf.53b8dc83db0defce0e4fb5cd63d7ee39.jpg"
# test_img_path="/prj/nowage/ImgAugWithSR/a4.jpg"
# img=superResolutionDo(test_img_path)
# imsave('/prj/nowage/ImgAugWithSR/a4x.jpg',img)

In [3]:
basePath="/prj/nowage/ImgAugWithSR/imgs/HardHatSample__small/"
images_path         = basePath + "images/"
images_super_path   = basePath + "images_super/"
labels_path         = basePath + "labels/"
labels_super_path   = basePath + "labels_super/"
images_resized_path = basePath + "images_resized/"
images_chopped_path = basePath + "images_chopped/"
labels_chopped_path = basePath + "labels_chopped/"
labels_resized_path = basePath + "labels_resized/"
# !ls {images_path}

In [4]:
def superResolutionBatch(images_path, images_super_path):
    file_list = os.listdir(images_path)
    for f in file_list:
        img=superResolutionDo(images_path+f)
        print(images_path+f)
        imsave(images_super_path+f,img)
        
superResolutionBatch( images_resized_path, images_super_path )  
!ls {images_super_path}

2021-11-11 13:27:56.753027: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-11 13:27:57.402317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 84 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1
2021-11-11 13:27:57.402702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5069 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0, compute capability: 6.1
2021-11-11 13:28:00.296169: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capabili

/prj/nowage/ImgAugWithSR/imgs/HardHatSample__small/images_resized/000003_jpg.rf.da4ac3b48b7a600add2281eacaf7dd82.jpg


2021-11-11 13:37:15.593506: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2021-11-11 13:37:15.593584: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2021-11-11 13:37:15.595593: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 84 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1
2021-11-11 13:37:15.595869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5069 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0, compute capability: 6.1
2021-11-11 13:37:15.648617: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1953 nodes (1608), 3017 edges (2

/prj/nowage/ImgAugWithSR/imgs/HardHatSample__small/images_resized/000002_jpg.rf.14c099486ff0a2df2d20d82da93b46f5.jpg


2021-11-11 13:46:36.183518: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2021-11-11 13:46:36.183589: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2021-11-11 13:46:36.185618: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 84 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1
2021-11-11 13:46:36.185891: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5069 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0, compute capability: 6.1
2021-11-11 13:46:36.238459: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1953 nodes (1608), 3017 edges (2

/prj/nowage/ImgAugWithSR/imgs/HardHatSample__small/images_resized/000004_jpg.rf.8f0984ca266ad0bb1ccce4a3dc095291.jpg


2021-11-11 13:56:44.098299: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2021-11-11 13:56:44.098371: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2021-11-11 13:56:44.100308: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 84 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1
2021-11-11 13:56:44.100587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5069 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0, compute capability: 6.1
2021-11-11 13:56:44.156979: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1953 nodes (1608), 3017 edges (2

/prj/nowage/ImgAugWithSR/imgs/HardHatSample__small/images_resized/000001_jpg.rf.45fd629cae669b7236f4f5484605be64.jpg
000001_jpg.rf.45fd629cae669b7236f4f5484605be64.jpg
000002_jpg.rf.14c099486ff0a2df2d20d82da93b46f5.jpg
000003_jpg.rf.da4ac3b48b7a600add2281eacaf7dd82.jpg
000004_jpg.rf.8f0984ca266ad0bb1ccce4a3dc095291.jpg
